## Display multi-profiles data

This script aims at being a sandbox (and only a sandbox) to display multi-profile files data  ({wmo}_prof.nc and {wmo}_Sprof.nc Argo files) 
and allows some investigations on anomalies

Author: D.Dobler (Euro-Argo ERIC)

Date: 30 September 2024

Version: 1.0

<!--TABLE OF CONTENTS-->
Contents:
  - [Display multi-profiles data](#Display-multi-profiles-data)
  - [I - Librairies](#I---Librairies)
  - [II - Some configuration](#II---Some-configuration)
  - [III - Functions](#III---Functions)
  - [IV - Analyses on whether adjustment was applied on pressure](#IV---Analyses-on-whether-adjustment-was-applied-on-pressure)
  - [V - Extract data](#V---Extract-data)
  - [VI - Plot section chart and overlaid profiles (and some filtering)](#VI---Plot-section-chart-and-overlaid-profiles-(and-some-filtering))
  - [VII - Save plots by cycle for CORE parameters](#VII---Save-plots-by-cycle-for-CORE-parameters)
  - [VIII - Plot for a subset of cycles - specific analyses](#VIII---Plot-for-a-subset-of-cycles---specific-analyses)
    - [VIII.1 For RAW parameters](#VIII.1-For-RAW-parameters)
    - [VIII.2 For ADJUSTED parameters](#VIII.2-For-ADJUSTED-parameters)
  - [IX - Extract values and information for specific QC + DMQC report preparation](#IX---Extract-values-and-information-for-specific-QC-+-DMQC-report-preparation)
  - [X - Transient dirt analysis](#X---Transient-dirt-analysis)

## I - Librairies

In [ ]:
import requests
import os
import numpy as np
from netCDF4 import Dataset
import datetime as dt
import shutil
import matplotlib.pyplot    as plt
import pandas as pd
import gsw
import sys
import cmocean
import array

sys.set_int_max_str_digits(8000)

## II - Some configuration

In [ ]:
# Colors for QC
qc_colors=np.zeros((10,3))

qc_colors[1,:]=[0.1953, 0.8008, 0.1953] # QC1 "green" 
qc_colors[2,:]=[0.9769, 0.9839, 0.0805] # QC2 "yellow"
qc_colors[3,:]=[1.0000, 0.6470,      0] # QC3 "orange"
qc_colors[4,:]=[1.0000,      0,      0] # QC4 "red"
qc_colors[5,:]=[0.1250, 0.6953, 0.6641] # QC5 "light green"	
#qc_colors[8,:]=[1.0000, 0.5469,      0] # QC8 "dark orange"	
qc_colors[8,:]=[0.5000,      0, 1.0000] # QC8 "violet"	

mode_colors=np.zeros((4,3))
mode_colors[1,:]=[1.0000,      0,      0] # data_mode R "red"
mode_colors[2,:]=[1.0000, 0.6470,      0] # data_mode A "orange"
mode_colors[3,:]=[0.1953, 0.8008, 0.1953] # data_mode D "green" 

# Choose where to save you temporary files
wrk_dir="C:/Users/ddobler/Documents/08_DD_scripts/02_Scripts_Prof_Data_display/Disp_Prof_WorkDir/"

## III - Functions

In [ ]:
def qc_from_char2int(val):

    shape_ini=val.shape
    tmp=((np.array(val)).astype('|S1')).tobytes().decode()
    tmp=tmp.replace(' ','6') # beware: this is only for computational issue, QC6 is unused usually
    tmp=list(tmp)
    out_val=np.array(tmp,dtype='int')
    out_val=np.reshape(out_val,shape_ini)
    return out_val

In [ ]:
def dir_from_char2str(val):
    out_val=np.array(list((np.array(val)).tobytes().decode()))
    return out_val

In [ ]:
def get_station_parameters_indices(ds):
    # function get parameter data mode.
    
    shape_ini=np.shape(ds.variables['STATION_PARAMETERS'][:])
    STATION_PARAMETERS=dir_from_char2str(ds.variables['STATION_PARAMETERS'][:])
    STATION_PARAMETERS_3D=np.reshape(STATION_PARAMETERS,(shape_ini[0],shape_ini[1],shape_ini[2]))
    n_prof=shape_ini[0]
    n_param=shape_ini[1]
    dict_param={}

    # handle case when all cycles and especially the first does not contain all parameters
    l_param=[None] * (n_param*n_prof)
    i=0
    for iprof in range(n_prof):
        for iparam in range(n_param):
            param_name=''.join(STATION_PARAMETERS_3D[iprof,iparam,:]).strip()
            l_param[i]=param_name
            i=i+1

    l_param=np.unique(l_param)
    print(l_param)
    
    for param_name in l_param:
        print("New param_name:", param_name)
        if len(param_name) > 0:
            for ii in range(len(param_name)):
                #print(param_name[ii])
                if ii==0:
                    index = (STATION_PARAMETERS_3D[:,:,ii]==param_name[ii])
                else:
                    index = index & (STATION_PARAMETERS_3D[:,:,ii]==param_name[ii])
        
            #print(np.where(index))
            dict_param[param_name]=np.where(index)
        
    #print(index)
    
    #print(dict_param["PRES"])
    
    #print(STATION_PARAMETERS_3D[:,6,:])

    return dict_param

In [ ]:
def transfer_mode_from_str2int(v0_data_mode):
    v0_data_mode_str=dir_from_char2str(v0_data_mode)
    v0_data_mode_int=np.zeros(len(v0_data_mode_str))
    v0_data_mode_int[np.where(v0_data_mode_str=='R')]=1
    v0_data_mode_int[np.where(v0_data_mode_str=='A')]=2
    v0_data_mode_int[np.where(v0_data_mode_str=='D')]=3
    return v0_data_mode_int

In [ ]:
def moving_average_1D(data,ns=5):
    data_smooth=np.zeros(data.shape)
    data_smooth[ns:-ns]=np.convolve(data, np.ones((ns*2+1,))/(ns*2+1), mode='valid')
    data_smooth[:ns]=data_smooth[ns]
    data_smooth[-ns:]=data_smooth[-ns-1]

    return data_smooth

In [ ]:
def plot_PTSR(P,T,S,R,C,index,wmo,adj,index_text):
    # P : pressure vector
    # T : temperature vector
    # S : salinity vector
    # R : density anomaly vector
    # C : cycle number
    # index: subset to plot
    # wmo: float wmo_id
    # adj="RAW" or "ADJ"
    ic_index=(C[index]).astype('int')
    t_index=T[index]
    s_index=S[index]
    p_index=P[index]
    r_index=R[index]
    mycmap=plt.cm.jet(np.linspace(0,1,ic_index.max()+1))
    #print(mycmap)
    
    plt.figure(figsize=(15,5))
    
    plt.subplot(1,3,1)
    for ic in np.unique(ic_index):
        plt.plot(t_index[ic_index==ic],-p_index[ic_index==ic],color=mycmap[ic])
    plt.grid()
    plt.title(wmo+ ' TEMP_' + adj + ' (' + index_text + ')')
    
    plt.subplot(1,3,2)
    for ic in np.unique(ic_index):
        plt.plot(s_index[ic_index==ic],-p_index[ic_index==ic],color=mycmap[ic])
    plt.grid()
    plt.title(wmo+ ' PSAL_' + adj + ' (' + index_text + ')')
    
    plt.subplot(1,3,3)
    for ic in np.unique(ic_index):
        plt.plot(r_index[ic_index==ic],-p_index[ic_index==ic],color=mycmap[ic])
    plt.grid()    
    plt.title(wmo+ ' absolute density anomaly ' + adj)

    return plt

In [ ]:
def plot_PTSR_1cycle(P,T,S,R,C,index,i_ok_bg,wmo,adj,index_text):
    # P : pressure vector
    # T : temperature vector
    # S : salinity vector
    # R : density anomaly vector
    # C : cycle number
    # index: subset to plot
    # wmo: float wmo_id
    # adj="RAW" or "ADJ"
    grey_shade=(0.75,0.75,0.75)

    #icok_index=(C[i_ok_bg]).astype('int')
    #t_index=T[icok_index]
    #s_index=S[icok_index]
    #p_index=P[icok_index]
    #r_index=R[icok_index]
    
    plt.figure(figsize=(15,5))
    
    plt.subplot(1,3,1)
    plt.scatter(T[i_ok_bg],-P[i_ok_bg],color=grey_shade)
    plt.plot(T[index],-P[index],color='b')
    plt.grid()
    plt.title(' TEMP_' + adj )
    
    plt.subplot(1,3,2)
    plt.scatter(S[i_ok_bg],-P[i_ok_bg],color=grey_shade)
    plt.plot(S[index],-P[index],color='b')
    plt.grid()
    plt.title(' PSAL_' + adj )

    plt.subplot(1,3,3)
    plt.scatter(R[i_ok_bg],-P[i_ok_bg],color=grey_shade)
    plt.plot(R[index],-P[index],color='b')
    plt.grid()
    plt.title(' absolute density anomaly ' + adj)

    cycle=np.unique(C[index])
    cycle=str(int(cycle[0]))
    plt.suptitle(wmo+' CYCLE ' + cycle)

    return plt

In [ ]:
def get_file_on_http(file_type,wrk_dir,dac,wmo):
    if not os.path.exists(wrk_dir):
        os.mkdir(wrk_dir)
    if file_type == "BGC"  : suffixe="_Sprof.nc"
    if file_type == "CORE" : suffixe="_prof.nc"
    if file_type == "TRAJ" : suffixe="_BRtraj.nc"
    file=wrk_dir + wmo + suffixe
    URL = "https://data-argo.ifremer.fr/dac/"+dac+"/"+wmo+"/"+wmo+suffixe
    response = requests.get(URL)
    open(file, "wb").write(response.content)
    return file
    

## IV - Analyses on whether adjustment was applied on pressure

In [ ]:
l_wmo=['1900517','1900518','1901350','1901351','3901085','3901086']
dac='coriolis'
for wmo in l_wmo:
    file=get_file_on_http('CORE',wrk_dir,'coriolis',wmo)
    
    ds=Dataset(file,'r')
    ds.set_auto_mask(False) # to avoid masked array, a little bit more tricky to manage
    
    PF=ds.variables['PLATFORM_NUMBER'][:].tobytes().decode().strip()
    lon=ds.variables['LONGITUDE'][:]
    lat=ds.variables['LATITUDE'][:]
    JULD=ds.variables['JULD'][:]
    JULD_QC=ds.variables['JULD_QC'][:]
    POSITION_QC=ds.variables['POSITION_QC'][:]
    JULD_QC=qc_from_char2int(JULD_QC)
    POSITION_QC=qc_from_char2int(POSITION_QC)
    #print(JULD_QC)
    #print(POSITION_QC)
    
    PRES=ds.variables['PRES'][:]
    PRES_ADJUSTED=ds.variables['PRES_ADJUSTED'][:]
    PRES[np.where(PRES>=99999)]=np.nan
    PRES_ADJUSTED[np.where(PRES_ADJUSTED>=99999)]=np.nan
    dPRES=PRES-PRES_ADJUSTED
    print(wmo,np.nanmin(dPRES),np.nanmax(dPRES),np.nanmean(dPRES))

## V - Extract data

In [ ]:
# Choose you WMO (indicate the DAC, and the type of data: CORE for temp, psal, pres ({wmo}_prof.nc file) or BGC for BGC parameters ({wmo}_Sprof.nc file)
dac,wmo,type='bodc','3902398','CORE'
dac,wmo,type='coriolis','6902923','CORE'
dac,wmo,type='coriolis','6903091','BGC'
dac,wmo,type='aoml','1901379','BGC'
dac,wmo,type='bodc','3901578','TRAJ'
dac,wmo,type='coriolis','3902014','CORE'
dac,wmo,type='coriolis','5906972','BGC'
dac,wmo,type='aoml','4903220','CORE'
#dac,wmo,type='aoml','4903217','CORE'


file=get_file_on_http(type,wrk_dir,dac,wmo)

ds=Dataset(file,'r')

ds.set_auto_mask(False) # to avoid masked array, a little bit more tricky to manage

PF=ds.variables['PLATFORM_NUMBER'][:].tobytes().decode().strip()
lon=ds.variables['LONGITUDE'][:]
lat=ds.variables['LATITUDE'][:]
JULD=ds.variables['JULD'][:]
JULD_QC=ds.variables['JULD_QC'][:]
POSITION_QC=ds.variables['POSITION_QC'][:]
JULD_QC=qc_from_char2int(JULD_QC)
POSITION_QC=qc_from_char2int(POSITION_QC)
#print(JULD_QC)
#print(POSITION_QC)

PRES=ds.variables['PRES'][:]

JULD[np.where(JULD>=99999)]=np.nan
lat[np.where(lat>=99999)]=np.nan
lon[np.where(lon>=99999)]=np.nan
PRES[np.where(PRES>=99999)]=np.nan
print("JULD_min,JULD_max,latmin,latmax,lonmin,lonmax,pres_min,pres_max")
print(np.nanmin(JULD),np.nanmax(JULD),np.nanmin(lat),np.nanmax(lat),np.nanmin(lon),np.nanmax(lon),np.nanmin(PRES),np.nanmax(PRES))

#print(PRES[1,:])
if type == "CORE" :
    PRES_QC=ds.variables['PRES_QC'][:,:]

DIRECTION = ds.variables['DIRECTION'][:]
DIRECTION = dir_from_char2str(DIRECTION)
tmp,DIRECTION_2=np.meshgrid(PRES[0,:],DIRECTION)

if type == "CORE" :
    DATA_MODE_ini=ds.variables['DATA_MODE'][:]
else:
    DATA_MODE_ini=ds.variables['PARAMETER_DATA_MODE'][:]
data_mode_ini_shape=np.shape(DATA_MODE_ini)
DATA_MODE = dir_from_char2str(DATA_MODE_ini).tobytes().decode().strip()
#print(data_mode_ini_shape)

i_ADJ=np.array(np.where((DATA_MODE=='A')|(DATA_MODE=='D'))[:])
adj_present=(i_ADJ.size>0)
print("Is there adjusted values ? ",adj_present)

data_mode_param_indices_dict_by_param=get_station_parameters_indices(ds)
#print(data_mode_param_indices_dict_by_param)

if type == "CORE" :
    sampling_scheme=ds.variables['VERTICAL_SAMPLING_SCHEME'][:]
    i_primary=[]
    primary=np.zeros(DIRECTION_2.shape)
    for isampl in range(sampling_scheme.shape[0]):
        l_sampling_scheme = sampling_scheme[isampl,:].tobytes().decode().strip()
        #print(l_sampling_scheme)
        if (l_sampling_scheme[:7]=="Primary"):
            i_primary=np.concatenate((list(i_primary),[isampl]))
    i_primary=i_primary.astype('int')
    primary[i_primary,:]=1
else:
    primary=np.ones(DIRECTION_2.shape)

i_KO=np.where((DIRECTION_2=='D') | (PRES>=99999) | (primary==0))
i_OK=np.where((DIRECTION_2=='A') & ~(PRES>=99999) & (primary==1))

#i_KO=np.where((PRES>=99999) )
#i_OK=np.where(~(PRES>=99999) )

#i_KO=np.where((PRES>=99999) | (primary==0))
#print("DIRECTION_2[i_KO]")
#print(DIRECTION_2[i_KO])
#print("PRES[i_KO]")
#print(PRES[i_KO])
#print("primary[i_KO]")
#print(primary[i_KO])

print("Number of Descending profiles:")
N_prof=DIRECTION.shape[0]
N_desc=np.array(np.where(DIRECTION=='D')).size
print(N_desc,"(",100*N_desc/N_prof,"%)")

print("Number of Primary profiles:")
N_prof=DIRECTION.shape[0]
N_prim=np.array(np.where(primary[:,0]==1)).size
print(N_prim,"(",100*N_prim/N_prof,"%)")

CYCLE = ds.variables['CYCLE_NUMBER'][:]
tmp,CYCLE_2=np.array(np.meshgrid(PRES[0,:],CYCLE))
CYCLE =(np.array(CYCLE_2)).astype('float')


tmp,lon_2=np.array(np.meshgrid(PRES[0,:],lon))
lon=np.array(lon_2)
tmp,lat_2=np.array(np.meshgrid(PRES[0,:],lat))
lat=np.array(lat_2)
tmp,JULD_2=np.array(np.meshgrid(PRES[0,:],JULD))
JULD=np.array(JULD_2)
JULD=JULD.astype('float')
ref_date=np.datetime64("1950-01-01T00:00:00")

prof_date_dt=JULD*86400*np.timedelta64(1, 's')+ref_date
#print(JULD)
#print(prof_date_dt)

#lon, lat, JULD, CYCLE=lon[i_OK], lat[i_OK], JULD[i_OK],CYCLE[i_OK]
lon[i_KO], lat[i_KO], JULD[i_KO]=np.nan, np.nan, np.nan
print("Number of ignored points (Descending or PRES masked or not primary sampling): ", np.array(i_KO).size)
PRES[i_KO]=np.nan

if type == "CORE" :
    PSAL=np.array(ds.variables['PSAL'][:,:])
    PSAL_QC=np.array(ds.variables['PSAL_QC'][:,:])
    TEMP=np.array(ds.variables['TEMP'][:,:])
    TEMP_QC=np.array(ds.variables['TEMP_QC'][:,:])
    
    PSAL_QC=qc_from_char2int(PSAL_QC)
    PRES_QC=qc_from_char2int(PRES_QC)
    TEMP_QC=qc_from_char2int(TEMP_QC)
    
    #PRES, PRES_QC=PRES[i_OK], PRES_QC[i_OK]
    #PSAL, PSAL_QC=PSAL[i_OK], PSAL_QC[i_OK]
    #TEMP, TEMP_QC=TEMP[i_OK], TEMP_QC[i_OK]
    
    PRES[i_KO]=np.nan
    PSAL[i_KO]=np.nan
    TEMP[i_KO]=np.nan
    
    
    CTD_QC=np.concatenate(([PSAL_QC],[TEMP_QC],[PRES_QC]))
    RHO_QC=np.nanmax(CTD_QC,axis=0)
    
    
    
    
    TEMP_ADJUSTED=np.array(ds.variables['TEMP_ADJUSTED'][:,:])
    TEMP_ADJUSTED_QC=np.array(ds.variables['TEMP_ADJUSTED_QC'][:,:])
    PRES_ADJUSTED=np.array(ds.variables['PRES_ADJUSTED'][:,:])
    PRES_ADJUSTED_QC=np.array(ds.variables['PRES_ADJUSTED_QC'][:,:])
    PSAL_ADJUSTED=np.array(ds.variables['PSAL_ADJUSTED'][:,:])
    PSAL_ADJUSTED_QC=np.array(ds.variables['PSAL_ADJUSTED_QC'][:,:])
    PSAL_ADJUSTED_QC=qc_from_char2int(PSAL_ADJUSTED_QC)
    PRES_ADJUSTED_QC=qc_from_char2int(PRES_ADJUSTED_QC)
    TEMP_ADJUSTED_QC=qc_from_char2int(TEMP_ADJUSTED_QC)
    
    #PRES_ADJUSTED, PRES_ADJUSTED_QC=PRES_ADJUSTED[i_OK], PRES_ADJUSTED_QC[i_OK]
    #PSAL_ADJUSTED, PSAL_ADJUSTED_QC=PSAL_ADJUSTED[i_OK], PSAL_ADJUSTED_QC[i_OK]
    #TEMP_ADJUSTED, TEMP_ADJUSTED_QC=TEMP_ADJUSTED[i_OK], TEMP_ADJUSTED_QC[i_OK]
    PRES_ADJUSTED[i_KO]=np.nan
    PSAL_ADJUSTED[i_KO]=np.nan
    TEMP_ADJUSTED[i_KO]=np.nan
    
    CTD_ADJUSTED_QC=np.concatenate(([PSAL_ADJUSTED_QC],[TEMP_ADJUSTED_QC],[PRES_ADJUSTED_QC]))
    RHO_ADJUSTED_QC=np.nanmax(CTD_ADJUSTED_QC,axis=0)
    
    
    
    
    #ds.close()
    
    SA=gsw.SA_from_SP(PSAL, PRES, lon, lat)
    RHO=gsw.pot_rho_t_exact(SA, TEMP, PRES, 0)-1000
    
    
    SA_ADJUSTED=gsw.SA_from_SP(PSAL_ADJUSTED, PRES_ADJUSTED, lon, lat)
    RHO_ADJUSTED=gsw.pot_rho_t_exact(SA_ADJUSTED, TEMP_ADJUSTED, PRES_ADJUSTED, 0)-1000

## VI - Plot section chart and overlaid profiles (and some filtering)

In [ ]:
# choose your plot type "scatter" | "tripcolor" | "pcolormesh'
section_plot_type= "scatter" 
lmarkersize=60 #30
lqcmarkersize=3
loverlaymarkersize=6
# choose your x-axis "prof_date_dt" | "CYCLE"
x_axis_type = "prof_date_dt" 
if x_axis_type == "prof_date_dt":
    x_axis=prof_date_dt 
if x_axis_type == "CYCLE":
    x_axis=CYCLE

# choose the parameter to display (a few examples below):
#l_param=["BBP700","CDOM","DOWN_IRRADIANCE490","CHLA","CHLA_ADJUSTED","DOXY","NITRATE_ADJUSTED","PH_IN_SITU_TOTAL"]
#l_param=["NITRATE","NITRATE_ADJUSTED"]
#l_param=["DOXY"]
#if adj_present:
#    l_param=["PRES","PSAL","TEMP","RHO","PSAL_ADJUSTED","TEMP_ADJUSTED","RHO_ADJUSTED"]
#else:
#    l_param=["PRES","PSAL","TEMP","RHO"]
l_param=["PSAL","TEMP","RHO"]


for param in l_param:
    
    print("")
    print("Treating " + param)
    print("")

    if (param in ["RHO"]) & (type == "CORE"):
        v0=RHO
        v0_QC=RHO_QC
    elif (param in ["RHO_ADJUSTED"]) & (type == "CORE"):
        v0=RHO_ADJUSTED
        v0_QC=RHO_ADJUSTED_QC
    elif param not in ["RHO","RHO_ADJUSTED"]:
        v0=np.array(ds.variables[param][:,:])
        v0_QC=np.array(ds.variables[param + '_QC'][:,:])
        v0_QC=qc_from_char2int(v0_QC)
        v0[i_KO]=np.nan
        PRES[i_KO]=np.nan
    
    z0=PRES

    # Choose the colormap depending on parameter and v0 scale
    if param in ["PRES","PRES_ADJUSTED"]:
        l_cmap=cmocean.cm.deep
    elif param in ["PSAL","PSAL_ADJUSTED"]:
        l_cmap=cmocean.cm.haline
    elif param in ["TEMP","TEMP_ADJUSTED"]:
        l_cmap=cmocean.cm.thermal
    elif param in ["RHO","RHO_ADJUSTED"]:
        l_cmap='jet'
        l_cmap="rho"
        l_cmap=cmocean.cm.dense   
    elif param in ["BBP700","BBP700_ADJUSTED"]:
        v0=v0*1e5
        l_cmap=cmocean.cm.matter
    elif param=="CDOM":
        l_cmap=cmocean.cm.matter
    elif param in ["DOXY","DOXY_ADJUSTED"]:
        l_cmap='spring_r'
        l_cmap=cmocean.cm.oxy
    elif param in ["CHLA","CHLA_ADJUSTED"]:
        v0=np.log(v0)
        l_cmap=cmocean.cm.algae
    elif param in ["DOWN_IRRADIANCE380","DOWN_IRRADIANCE412","DOWN_IRRADIANCE490","DOWNWELLING_PAR"]:
        l_cmap=cmocean.cm.solar
    elif param in ["NITRATE","NITRATE_ADJUSTED"]:
        l_cmap='autumn_r'
        #l_cmap='jet'
    elif param in ["PH_IN_SITU_TOTAL","PH_IN_SITU_TOTAL_ADJUSTED"]:
        l_cmap='jet_r'
    else:
        l_cmap='jet'

    # Choose the percentile for the colorscale
    if param in ["BBP700","BBP700_ADJUSTED"]:
        percent=98
    else:
        percent=95        

    if param not in ["RHO","RHO_ADJUSTED"]:
        if type == "CORE":
            v0_data_mode=DATA_MODE_ini[data_mode_param_indices_dict_by_param[param][0]]
        else:
            v0_data_mode=DATA_MODE_ini[data_mode_param_indices_dict_by_param[param]]
        x_axis_data_mode=x_axis[data_mode_param_indices_dict_by_param[param][0]]
        #print(v0_data_mode.tobytes().decode().strip())
        v0_data_mode_int=transfer_mode_from_str2int(v0_data_mode)
        #print(v0_data_mode_int)
        #print(x_axis_data_mode)

    unique_data_mode_list=np.unique(v0_data_mode_int)

    unique_qc_list=np.unique(v0_QC)
    i_QC_good=[]
    print("param=",param)
    if param=="CDOM":
        print("special QC list for CDOM")
        good_text=" QC in (0,1,2,5,8) "
        i_QC_good=np.where((v0_QC==0) |(v0_QC==1) | (v0_QC==2) | (v0_QC==5) | (v0_QC==8))
        i_QC_notmissing=np.where((v0_QC==0) |(v0_QC==1) | (v0_QC==2) | (v0_QC==3) | (v0_QC==4)| (v0_QC==5) | (v0_QC==8))
        #print("i_QC_good=",i_QC_good)
        #print("i_QC_notmissing=",i_QC_notmissing)
    elif param in ["DOXY","DOXY_ADJUSTED","CHLA","CHLA_ADJUSTED","NITRATE","NITRATE_ADJUSTED",
                   "PH_IN_SITU_TOTAL","PH_IN_SITU_TOTAL_ADJUSTED","PSAL","TEMP","RHO"]:
        print("special QC list for DOXY, CHLA, NITRATE and pH")
        good_text=" QC in (1,2,3,5,8) "
        i_QC_good=np.where((v0_QC==1) | (v0_QC==2) | (v0_QC==3) | (v0_QC==5) | (v0_QC==8))
        i_QC_notmissing=np.where((v0_QC==1) | (v0_QC==2) | (v0_QC==3) | (v0_QC==4)| (v0_QC==5) | (v0_QC==8))
        #print("i_QC_good=",i_QC_good)
        #print("i_QC_notmissing=",i_QC_notmissing)
    else:
        good_text=" QC in (1,2,5,8) "
        i_QC_good=np.where((v0_QC==1) | (v0_QC==2) | (v0_QC==5) | (v0_QC==8))
        i_QC_notmissing=np.where((v0_QC==1) | (v0_QC==2) | (v0_QC==3) | (v0_QC==4)| (v0_QC==5) | (v0_QC==8))
        #print("i_QC_good=",i_QC_good)
        #print("i_QC_notmissing=",i_QC_notmissing)

    i_QC_bad=np.where((v0_QC==0) | (v0_QC==3) | (v0_QC==4) | (v0_QC==9))

    v0_good=v0[i_QC_good]
    v0_QC_good=v0_QC[i_QC_good]
    unique_qc_good_list=np.unique(v0_QC_good)
    print("v0_good")
    print(v0_good)
    
    v0_notmissing=v0[i_QC_notmissing]
    v0_QC_notmissing=v0_QC[i_QC_notmissing]
    unique_qc_notmissing_list=np.unique(v0_QC_notmissing)


    VMIN=np.nanpercentile(v0_good,100-percent)
    VMAX=np.nanpercentile(v0_good,percent)
    print("VMIN,VMAX,np.nanmin(v0_good),np.nanmax(v0_good)")
    print(VMIN,VMAX,np.nanmin(v0_good),np.nanmax(v0_good))
    VMIN_CYCLE=np.min(CYCLE)
    VMAX_CYCLE=np.max(CYCLE)

    
    if section_plot_type == "pcolormesh": # This does not work well

        y_axis=np.zeros(PRES.shape)
        y_axis[:]=PRES[:]
        y_axis[np.where(PRES>=99999)]=np.max(PRES)
        iy_max=np.max(np.where(PRES<99999)[1])
        print("iy_max=",iy_max)

        c_m_good = np.zeros(v0.shape)
        c_m_good[:] = v0[:]
        c_m_good = np.ma.masked_where(v0>=99999,c_m_good)
        c_m_good = np.ma.masked_where(((v0_QC==0) | (v0_QC==3) | (v0_QC==4) | (v0_QC==9)),c_m_good)

        c_m_notmissing = np.zeros(v0.shape)
        c_m_notmissing[:] = v0[:]
        c_m_notmissing = np.ma.masked_where(v0>=99999,c_m_notmissing)
        c_m_notmissing = np.ma.masked_where(((v0_QC==0) | (v0_QC==9)),c_m_notmissing)


    plt.figure(figsize=(25,5))
    
    plt.subplot(1,4,1)
    plt.title("Section Chart all qc " + wmo + " " + param)
    if section_plot_type == "scatter":
        plt.scatter(x_axis[i_QC_notmissing],-z0[i_QC_notmissing],c=v0[i_QC_notmissing],s=lmarkersize,cmap=l_cmap,marker='.',vmin=VMIN,vmax=VMAX)
        print(x_axis[i_QC_notmissing])
    if section_plot_type == "tripcolor":
        plt.tripcolor(x_axis[i_QC_notmissing],-z0[i_QC_notmissing],v0[i_QC_notmissing],cmap=l_cmap,vmin=VMIN,vmax=VMAX)
    if section_plot_type == "pcolormesh":
        plt.pcolormesh(x_axis[:,:iy_max],-y_axis[:,:iy_max],c_m_notmissing[:,:iy_max],cmap=l_cmap,vmin=VMIN,vmax=VMAX, shading='nearest')
        #plt.ylim(-2000,0)
    plt.colorbar(extend='both')
    plt.grid()
    if x_axis_type == "prof_date_dt": plt.xticks(rotation=30)
    
    plt.subplot(1,4,2)
    plt.title(param + "_QC")
    for lqc in unique_qc_notmissing_list:
        print("lqc=",lqc)
        i_lqc=np.where(v0_QC==lqc)[:]
        plt.plot(x_axis[i_lqc], -z0[i_lqc],marker='.',markersize=lqcmarkersize,linestyle="none",c=qc_colors[int(lqc),:],label='QC'+str(lqc))
    plt.legend()
    plt.grid()
    if x_axis_type == "prof_date_dt": plt.xticks(rotation=30)
    
    plt.subplot(1,4,3)
    plt.title("Section Chart" + good_text + wmo + " " + param)
    if section_plot_type == "scatter":
        plt.scatter(x_axis[i_QC_good],-z0[i_QC_good],c=v0[i_QC_good],s=lmarkersize,cmap=l_cmap,marker='.',vmin=VMIN,vmax=VMAX)
    if section_plot_type == "tripcolor":
        plt.tripcolor(x_axis[i_QC_good],-z0[i_QC_good],v0[i_QC_good],cmap=l_cmap,vmin=VMIN,vmax=VMAX)
    if section_plot_type == "pcolormesh":
        plt.pcolormesh(x_axis,-y_axis,c_m_good,cmap=l_cmap,vmin=VMIN,vmax=VMAX,shading='nearest')
    plt.colorbar(extend='both')
    plt.grid()
    if x_axis_type == "prof_date_dt": plt.xticks(rotation=30)

    plt.subplot(1,4,4)
    plt.title(param + "_QC")
    for lqc in unique_qc_good_list:
        i_lqc=np.where(v0_QC==lqc)[:]
        plt.plot(x_axis[i_lqc], -z0[i_lqc],marker='.',markersize=lqcmarkersize,linestyle="none",c=qc_colors[int(lqc),:],label='QC'+str(lqc))
    plt.legend()
    plt.grid()
    if x_axis_type == "prof_date_dt": plt.xticks(rotation=30)
    

    plt.figure(figsize=(25,1))

    for i in range(4):
        ax=plt.subplot(1,4,i+1)
        plt.title(param + "_DATA_MODE")
        for lmode in np.flipud(unique_data_mode_list):
            i_lmode=np.where(v0_data_mode_int==lmode)[:]
            plt.plot(x_axis_data_mode[i_lmode], v0_data_mode_int[i_lmode],c=mode_colors[int(lmode),:],marker='*',markersize=lqcmarkersize)
        plt.grid()
        ax.set_yticks((1,2,3))
        ax.set_yticklabels(("R", "A", "D"))
        plt.ylim(0.8,3.2)
        if x_axis_type == "prof_date_dt": plt.xticks(rotation=30)
    

    

    plt.figure(figsize=(25,5))
    
    plt.subplot(1,4,1)
    plt.title("Overlaid profiles all qc " + wmo + " " + param)
    plt.scatter(v0[i_QC_notmissing], -z0[i_QC_notmissing],c=CYCLE[i_QC_notmissing],cmap='jet',marker='.',s=loverlaymarkersize,vmin=VMIN_CYCLE,vmax=VMAX_CYCLE)
    plt.colorbar()
    plt.grid()
    
    plt.subplot(1,4,2)
    plt.title(param + "_QC")
    for lqc in unique_qc_notmissing_list:
        i_lqc=np.where(v0_QC==lqc)[:]
        plt.plot(v0[i_lqc], -z0[i_lqc],c=qc_colors[int(lqc),:],label='QC'+str(lqc),marker='.',markersize=lqcmarkersize,linestyle="none")
    plt.legend()
    plt.grid()

    plt.subplot(1,4,3)
    plt.title("Overlaid profiles" + good_text + wmo + " " + param)
    plt.scatter(v0[i_QC_good], -z0[i_QC_good],c=CYCLE[i_QC_good],cmap='jet',marker='.',s=loverlaymarkersize,vmin=VMIN_CYCLE,vmax=VMAX_CYCLE)
    plt.colorbar()
    plt.grid()
    
    plt.subplot(1,4,4)
    plt.title(param + "_QC")
    for lqc in unique_qc_good_list:
        i_lqc=np.where(v0_QC==lqc)[:]
        plt.plot(v0[i_lqc], -z0[i_lqc],c=qc_colors[int(lqc),:],label='QC'+str(lqc),marker='.',markersize=lqcmarkersize,linestyle="none")
    plt.legend()
    plt.grid()
    
    

## VII - Save plots by cycle for CORE parameters

In [ ]:
# plot and save all cycles together, then cycle by cycle with other cycles in grey shadings
if not os.path.exists(wrk_dir + "/"+wmo):
        os.mkdir(wrk_dir + "/"+wmo)

# Choose you filter
i_cycle=np.where((CYCLE>=CYCLE.min())&(CYCLE<=CYCLE.max())&(PSAL_QC!=4)&(PSAL_QC!=0)&(PSAL_QC!=9)&(TEMP_QC!=9)&(PSAL_QC!=6)&(TEMP_QC!=6))

plt=plot_PTSR(PRES,TEMP,PSAL,RHO,CYCLE,i_cycle,wmo,"RAW","PSAL RAW_QC != 0, 4, 9")
plt.savefig(wrk_dir + "/" + wmo + "/" + wmo + "_" + "all_cycles_psal_qc_12358.png")

for cycle in np.unique(CYCLE):
    print('Treating cycle {0:03d}'.format(int(cycle)))
    i_cycle=np.where((CYCLE==cycle)&(PSAL_QC!=4)&(PSAL_QC!=0)&(PSAL_QC!=9)&(TEMP_QC!=9)&(PSAL_QC!=6)&(TEMP_QC!=6))
    i_ok_bg=np.where((PSAL_QC!=4)&(PSAL_QC!=0)&(PSAL_QC!=9)&(TEMP_QC!=9)&(PSAL_QC!=6)&(TEMP_QC!=6))
    plot_PTSR_1cycle(PRES,TEMP,PSAL,RHO,CYCLE,i_cycle,i_ok_bg,wmo,"RAW","PSAL RAW_QC != 0, 4, 9")
    plt.savefig(wrk_dir + "/" + wmo + "/" + wmo + "_" + '{0:03d}'.format(int(cycle)) + "_psal_qc_12358.png")
    plt.close()

## VIII - Plot for a subset of cycles - specific analyses

### VIII.1 For RAW parameters

In [ ]:
# Plot subsets of cycles, at convenience
cy_min=CYCLE.min() # 10
cy_max=CYCLE.max() 
cy_min=142
cy_max=142


i_cycle=np.where((CYCLE>=cy_min)&(CYCLE<=cy_max))
plt=plot_PTSR(PRES,TEMP,PSAL,RHO,CYCLE,i_cycle,wmo,"RAW","all QC")
#plt.ylim(-750,-480)
#plt.xlim(26.4,26.8)

# The same wihtout QC_4 points (to see remaining issues)
i_cycle=np.where((CYCLE>=cy_min)&(CYCLE<=cy_max)&(PSAL_QC!=4)&(PSAL_QC!=0)&(PSAL_QC!=9)&(TEMP_QC!=9))
plt=plot_PTSR(PRES,TEMP,PSAL,RHO,CYCLE,i_cycle,wmo,"RAW","PSAL RAW_QC != 0, 4, 9")
#lt.subplot(1,3,2)
#plt.xlim(34.5,35.5)

i_cycle=np.where((CYCLE>=cy_min)&(CYCLE<=cy_max)&(TEMP_QC!=0)&(TEMP_QC!=4)&(PSAL_QC!=9)&(TEMP_QC!=9))
plt=plot_PTSR(PRES,TEMP,PSAL,RHO,CYCLE,i_cycle,wmo,"RAW","TEMP RAW_QC != 0 and 4")

i_cycle=np.where((CYCLE>=cy_min)&(CYCLE<=cy_max)&(PSAL_QC!=4)&(PSAL_QC!=0)&(TEMP_QC!=0)&(TEMP_QC!=4)&(PSAL_QC!=9)&(TEMP_QC!=9))
plt=plot_PTSR(PRES,TEMP,PSAL,RHO,CYCLE,i_cycle,wmo,"RAW","PSAL and TEMP RAW_QC != 0 and 4")

### VIII.2 For ADJUSTED parameters

In [ ]:
# Plot subsets of cycles, at convenience
cy_min=CYCLE.min() # 10
cy_max=CYCLE.max() 
cy_min=1
cy_max=1


i_cycle=np.where((CYCLE>=cy_min)&(CYCLE<=cy_max)& 
                 (PSAL_ADJUSTED_QC!=9)&(TEMP_ADJUSTED_QC!=9)&(PRES_ADJUSTED_QC!=9)&
                 (PSAL_ADJUSTED_QC!=6)&(TEMP_ADJUSTED_QC!=6)&(PRES_ADJUSTED_QC!=6))
plt=plot_PTSR(PRES_ADJUSTED,TEMP_ADJUSTED,PSAL_ADJUSTED,RHO_ADJUSTED,CYCLE,i_cycle,wmo,"ADJ","all QC")

# The same wihtout QC_4 points (to see remaining issues)
i_cycle=np.where((CYCLE>=cy_min)&(CYCLE<=cy_max)&
                 (PSAL_ADJUSTED_QC!=0)&(PSAL_ADJUSTED_QC!=4) & 
                 (PSAL_ADJUSTED_QC!=9)&(TEMP_ADJUSTED_QC!=9)&(PRES_ADJUSTED_QC!=9)&
                 (PSAL_ADJUSTED_QC!=6)&(TEMP_ADJUSTED_QC!=6)&(PRES_ADJUSTED_QC!=6))
plt=plot_PTSR(PRES_ADJUSTED,TEMP_ADJUSTED,PSAL_ADJUSTED,RHO_ADJUSTED,CYCLE,i_cycle,wmo,"ADJ","PSAL ADJ_QC != 0 and 4")

i_cycle=np.where((CYCLE>=cy_min)&(CYCLE<=cy_max)&
                 (TEMP_ADJUSTED_QC!=0)&(TEMP_ADJUSTED_QC!=4)& 
                 (PSAL_ADJUSTED_QC!=9)&(TEMP_ADJUSTED_QC!=9)&(PRES_ADJUSTED_QC!=9)&
                 (PSAL_ADJUSTED_QC!=6)&(TEMP_ADJUSTED_QC!=6)&(PRES_ADJUSTED_QC!=6))
plt=plot_PTSR(PRES_ADJUSTED,TEMP_ADJUSTED,PSAL_ADJUSTED,RHO_ADJUSTED,CYCLE,i_cycle,wmo,"ADJ","TEMP ADJ_QC != 0 and 4")

i_cycle=np.where((CYCLE>=cy_min)&(CYCLE<=cy_max)&
                 (PSAL_ADJUSTED_QC!=0)&(PSAL_ADJUSTED_QC!=4)& 
                 (TEMP_ADJUSTED_QC!=0)&(TEMP_ADJUSTED_QC!=4)& 
                 (PSAL_ADJUSTED_QC!=9)&(TEMP_ADJUSTED_QC!=9)&(PRES_ADJUSTED_QC!=9)&
                 (PSAL_ADJUSTED_QC!=6)&(TEMP_ADJUSTED_QC!=6)&(PRES_ADJUSTED_QC!=6))
plt=plot_PTSR(PRES_ADJUSTED,TEMP_ADJUSTED,PSAL_ADJUSTED,RHO_ADJUSTED,CYCLE,i_cycle,wmo,"ADJ","PSAL and TEMP ADJ_QC != 0 and 4")


## IX - Extract values and information for specific QC + DMQC report preparation

In [ ]:
# choose your criterion
level_ko=(PSAL<34.3)&(PRES<80)&(CYCLE==1) # &(PSAL_QC!=4)
level_ko=(PRES_QC==4)
level_ko=(PSAL_QC==4)|(TEMP_QC==4)


l_cycle=np.unique(CYCLE[np.where(level_ko)]);
# Prepare output for the DMQC report
for icycle in l_cycle:
    i_ko_1=np.where((PSAL_QC==4)&(TEMP_QC==4)&(CYCLE==icycle))
    i_ko_2=np.where((PSAL_QC==4)&(TEMP_QC!=4)&(CYCLE==icycle))
    if len(i_ko_1[0]) > 0:
        print(int(icycle),'& PSAL, TEMP & automatic QC & 1 & 4 & ',PRES[i_ko_1],' \\ \\')
    if len(i_ko_2[0]) > 0:
        print(int(icycle),'& PSAL & automatic QC & 1 & 4 & ',PRES[i_ko_2],' \\ \\')

# Display all values with respect to the criterion
print("PRES_QC=",PRES_QC[np.where(level_ko)])
print("PSAL=",PSAL[np.where(level_ko)])
print("PSAL_QC=",PSAL_QC[np.where(level_ko)])
print("TEMP=",TEMP[np.where(level_ko)])
print("TEMP_QC=",TEMP_QC[np.where(level_ko)])
print("RHO=",RHO[np.where(level_ko)])

## X - Transient dirt analysis
correlation betw. density anomaly and salinity vertical variations

In [ ]:
PSAL_smooth=np.zeros(PSAL.shape)
RHO_smooth=np.zeros(RHO.shape)
for i_cycle in np.unique(CYCLE):
    ii_cycle=np.where((CYCLE==i_cycle)& 
                      (PSAL_QC!=9)&(TEMP_QC!=9)&(PRES_QC!=9)&
                      (PSAL_QC!=6)&(TEMP_QC!=6)&(PRES_QC!=6))
    PSAL_smooth[ii_cycle]=moving_average_1D(PSAL[ii_cycle],ns=5)
    RHO_smooth[ii_cycle]=moving_average_1D(RHO[ii_cycle],ns=5)

PSAL_ano=PSAL-PSAL_smooth
RHO_ano=RHO-RHO_smooth
correl_1=(PSAL_ano-RHO_ano)/RHO_ano

# choose the cycle(s) to plot
cy_min=CYCLE.min() # 10
cy_max=CYCLE.max() 
cy_min=2
cy_max=2

i_cycle=np.where((CYCLE>=cy_min)&(CYCLE<=cy_max)& 
                 (PSAL_QC!=9)&(TEMP_QC!=9)&(PRES_QC!=9)&
                 (PSAL_QC!=6)&(TEMP_QC!=6)&(PRES_QC!=6))

plt.figure(figsize=(15,15))
plt.title(wmo+ ' deviation from mean value')
plt.plot(PSAL_ano[i_cycle],-PRES[i_cycle],label='PSAL_ano')
plt.plot(RHO_ano[i_cycle],-PRES[i_cycle],'r:',label='RHO_ano')
plt.grid()
plt.legend()

plt.figure(figsize=(15,15))
plt.plot(correl_1[i_cycle],-PRES[i_cycle])
plt.grid()